In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.metrics import confusion_matrix
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


###Uploading Data

In [2]:
df = pd.read_csv('/content/drive/MyDrive/detroit data/final_cleaned_detroit.csv')
df.head()

,Parcel ID,Taxpayer 1,Property Class,Total Floor Area,Total Acreage,Frontage,Depth,Building Count,Year Built,Sale Price,Assessed Value,Previous Assessed Value,Taxable Value,Previous Taxable Value,Neighborhood,fine_amount,yearly_average,vacant,Binary Tax Status,Binary Blight Violation,Binary Building Permit Status,Sale Date Year,Taxpayer City is Detroit,neighborhood_population,num_vacant_neighborhood,normcrime
0,21062470.,853 ASHLAND LLC,401.0,2468.0,0.079,30.0,115.0,1.0,1916.0,65000.0,61300.0,46200.0,61300.0,16361.0,Jefferson Chalmers,250.0,1602.2,0,1,1,0,2021.0,1,1695.0,49.0,0.945251
1,21063979.,"CASTLE, ROBERT M",401.0,1389.0,0.000,0.0,0.0,1.0,2002.0,13600.0,40100.0,31700.0,13286.0,12654.0,Morningside,0.0,3541.0,0,1,1,0,2014.0,0,3606.0,152.0,0.981974
2,21068429.,CITIMORTGAGE INC,401.0,1293.0,0.115,40.0,126.0,1.0,1938.0,79992.0,40400.0,33400.0,40400.0,15835.0,Moross-Morang,0.0,3200.6,0,1,1,0,2023.0,0,2610.0,85.0,1.226284
3,21069819.,"ROBINSON, DENISE",401.0,1540.0,0.105,40.0,114.0,1.0,1929.0,8912.0,39900.0,31300.0,14391.0,13706.0,Morningside,0.0,3541.0,0,1,1,0,2010.0,1,3606.0,152.0,0.981974
4,21070146.,"BRADFORD, WILLIAM",401.0,920.0,0.107,40.0,116.0,1.0,1938.0,20000.0,20300.0,16200.0,9396.0,8949.0,Outer Drive-Hayes,0.0,4247.8,0,1,1,0,1987.0,1,3234.0,304.0,1.313482


###Extra Cleaning-move to main

In [3]:
df = df.dropna(subset=["Previous Assessed Value"])
df = df.dropna(subset=["neighborhood_population"])
df = df.dropna(subset=["num_vacant_neighborhood"])
df = df.dropna(subset=["normcrime"])

In [4]:
df["fine_amount"] = df["fine_amount"].fillna(0)

###Randomly removing 10% of rows for final test

In [5]:
df_remaining = df

#dont use below for now
#df_10_percent = df.sample(frac=0.1, random_state=42)

#df_remaining = df.drop(df_10_percent.index)

In [6]:
#dont use for now
#X_test_final = df_10_percent[["Total Floor Area", "Total Acreage", "Frontage", "Depth", "Sale Price", "Taxpayer City is Detroit", "Binary Blight Violation", "Binary Building Permit Status", "fine_amount"]]
#y_test_final = df_10_percent["vacant"]

###Variable Selection and Train Test Split

In [7]:
#establishing x and y data
X = df_remaining[["Total Floor Area", "Total Acreage", "Frontage", "Depth", "Sale Price", "Taxpayer City is Detroit", "Binary Blight Violation", "Binary Building Permit Status", "fine_amount", "neighborhood_population", "num_vacant_neighborhood", "normcrime"]] #leaving out "Sale Date Year", "Year Built",
#X = df.drop(columns=["vacant", "Taxpayer 1", "Neighborhood", "Binary Blight Violation", "Parcel ID"])
y = df_remaining["vacant"]

In [8]:
#creeating randomized train/test split with random sectors of 41 (test=20% of data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [9]:
X_train.head(20)

,Total Floor Area,Total Acreage,Frontage,Depth,Sale Price,Taxpayer City is Detroit,Binary Blight Violation,Binary Building Permit Status,fine_amount,neighborhood_population,num_vacant_neighborhood,normcrime
146791,1877.0,0.149,50.0,130.0,70000.0,1,1,0,0.0,1309.0,8.0,0.865852
43885,1116.0,0.118,40.0,129.0,19000.0,1,1,1,0.0,1755.0,21.0,1.049231
169289,1355.0,0.101,37.0,118.0,4000.0,1,1,0,0.0,3204.0,30.0,1.169288
41578,1083.0,0.102,41.0,108.0,110000.0,1,1,0,0.0,904.0,20.0,1.162389
155146,860.0,0.206,80.0,112.0,0.0,1,1,0,0.0,2533.0,67.0,1.116068
22709,1371.0,0.085,35.0,106.0,3800.0,1,1,0,0.0,2980.0,182.0,1.133087
99860,1493.0,0.084,35.0,105.0,0.0,1,1,0,0.0,984.0,135.0,1.423577
116250,1165.0,0.124,45.0,120.0,32000.0,1,1,0,0.0,2002.0,18.0,1.460739
56017,976.0,0.120,44.0,120.0,12000.0,1,1,0,250.0,2231.0,107.0,0.971851
193542,1634.0,0.092,35.0,114.0,155818.0,0,1,0,250.0,1065.0,32.0,0.866479


In [10]:
y_train.head()
y_train.value_counts()

,count
vacant,
0,158959
1,10249


In [11]:
y_test.head()
y_test.value_counts()

,count
vacant,
0,39767
1,2536


###Sequential Deep Neural Network for Binary Classification with Dropout and Weights

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
import numpy as np


y_train_array = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train

#class weights which can be changed
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_array),
                                                  y=y_train_array)

#keras needs it to be a dictionary
class_weights = dict(enumerate(class_weights))
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)



#model parameters
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Set input shape to match your feature count
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

#compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

#run model
history = model.fit(X_train, y_train_array, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test), class_weight=class_weights, callbacks=[early_stopping])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5393 - loss: 0.7839 - val_accuracy: 0.6578 - val_loss: 0.6738
Epoch 2/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.6629 - loss: 0.6448 - val_accuracy: 0.7460 - val_loss: 0.5906
Epoch 3/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step - accuracy: 0.6764 - loss: 0.6321 - val_accuracy: 0.7116 - val_loss: 0.5912
Epoch 4/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.6680 - loss: 0.6312 - val_accuracy: 0.7358 - val_loss: 0.6109
Epoch 5/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.6762 - loss: 0.6212 - val_accuracy: 0.7240 - val_loss: 0.5817
Epoch 6/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - accuracy: 0.6690 - loss: 0.6152 - val_accuracy: 0.7550 - val_loss: 0.5171
Epoch 7/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.6719 - loss: 0.6162 - val_accuracy: 0.6449 - val_loss: 0.6063
Epoch 8/50
5288/5288 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.6598 - loss: 0

###Testing metrics: will add more here

In [13]:
len(X_test)

42303

In [18]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

1322/1322 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7565 - loss: 0.5173
Test Accuracy: 0.7549819350242615


###Matrix Results

In [25]:


predictions = model.predict(X_test)
predictions_binary = (predictions > 0.5).astype(int)  # Convert to binary labels

cm = confusion_matrix(y_test, predictions_binary)
print("Confusion Matrix:")
print(cm)

#[[TN, FP],
#[[FN, TP]]

1322/1322 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Confusion Matrix:
[[30473  9294]
 [ 1071  1465]]


In [26]:
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(f"Incorrectly predicted 'Vacant' (False Positives): {FP}")
print(f"Incorrectly predicted 'Not Vacant' (False Negatives): {FN}")

Incorrectly predicted 'Vacant' (False Positives): 9294
Incorrectly predicted 'Not Vacant' (False Negatives): 1071
